In [2]:


from typing import List, Dict
import json

# import Learning

from database_connect import client # gets MongoDB client, which gives access to data

Pinged your deployment. You successfully connected to MongoDB!


In [10]:
# constants

# default collection used

main_collection = "Section0"

In [7]:


# recursively create JSON
def obj_to_dict(obj):
    # recursive calls run depending on if the call's obj is a list, dictionary, object, or primitive ( int, string, float, etc)
    if isinstance(obj, list): # if list
        return [obj_to_dict(e) for e in obj]
    elif isinstance(obj, dict): # if dictionary or object
        return {key: obj_to_dict(value) for key, value in obj.items()}
    elif hasattr(obj, '__dict__'): # if object
        return {key: obj_to_dict(value) for key, value in obj.__dict__.items()}
    else: # if primitive
        return obj

# converts dictionary into student object
def dict_to_student(dict_data):
    # Initialize student Object
    student = Student(dict_data['name'], dict_data['grade'])

    # Initialize and fill subtopics
    for subtopic_data in dict_data['subtopics']:
        subtopic = Subtopic(subtopic_data['subtopic_name'], subtopic_data['grade'])
        subtopic.num_questions_answered = subtopic_data['num_questions_answered']

        metrics_data = subtopic_data['metrics']
        metrics = Metrics()

        for metric_name, metric_data in metrics_data.items():
            metric = Metric()
            metric.avg_score = metric_data['avg_score']
            metric.previous_scores = metric_data['previous_scores']

            if 'avg_time' in metric_data:
                metric.avg_time = metric_data['avg_time']
            if 'recent_times' in metric_data:
                metric.recent_times = metric_data['recent_times']
            if 'related_mistakes' in metric_data:
                metric.related_mistakes = metric_data['related_mistakes']

            setattr(metrics, metric_name, metric)

        subtopic.metrics = metrics
        student.subtopics.append(subtopic)

    return student

In [72]:
class StudentsCollection:
    def __init__(self, collection):
        database = client["Students"]
        self.name = collection
        self.collection = database[collection] # a collection ( ex: Section0)


    def add_student(self,student):
        # make student into dictionary format
        student_dict = student.in_dict_format()
        self.collection.insert_one(student_dict)
        print(f"{student.name} has been added to collection: {self.name}")

    # takes in student's name
    # returns wanted student from database
    def get_student(self, student_name):
        query = {"name": student_name}

        # Fetch the student data as a list
        student_dict_list = list(self.collection.find(query))

        try:
            if not student_dict_list:  # Checks if the list is empty
                raise ValueError("No student found with the given name.")

            # Get the first student_dict from the list (assuming there should only be one)
            student_dict = student_dict_list[0]
            # Do something with student_dict
            student_obj = dict_to_student(student_dict)
            print("Student found:", student_obj.name)
            return student_obj

        except ValueError as e:
            print(e)

    def delete_student(self,student):
        query = {"name": student.name}
        self.collection.delete_one(query)
        print(f"{student.name} has been deleted from collection: {self.name}")

    # gets list of all students in collection
    def current_student_names(self):
        student_names = []
        student_data = list(self.collection.find())

        for datam in student_data:

            student_names.append(datam["name"])
        return student_names



class Student:
    def __init__(self,name, grade: float):
        self.name = name
        self.grade = grade
        self.subtopics = [] # array of Subtopic objects


    def in_dict_format(self):
        return obj_to_dict(self)

class Subtopic:

    def __init__(self, subtopic_name, grade):
        # how many questions you answered for each of the 5 levels of a topic
        self.subtopic_name = subtopic_name
        self.grade = grade
        self.num_questions_answered = [0,0,0,0,0]
        self.metrics = Metrics()

class Metrics:

    def __init__(self):
        self.overall_avg = Metric("overall_avg")
        self.communication = Metric()
        self.interpretation = Metric()
        self.computation = Metric()
        self.conceptual = Metric()
        self.time = Metric("time")

class Metric:
    # special types: time, overall_avg
    def __init__(self,special_type = None):
        self.avg_score = 0
        self.previous_scores = []
        if special_type:
            if special_type == "time":
                self.avg_time = None
                self.recent_times = []
            if not special_type == "overall_avg":
                self.related_mistakes = []


In [76]:
student_collection =StudentsCollection(main_collection)

In [ ]:
# temporary helper functions
def manual_level_reset(name, sub_topic, level):
    # Get data from students.json
    data = get_ext_data(student_data_path)

    # Check if the student is in the database
    students = data['students']
    for student in students:
        if name in student:
            # Check if the student has a section for the given subtopic
            sections = student[name]
            for section in sections:
                if section['sub_topic'] == sub_topic:
                    # Reset the student's metrics
                    section['proficiency_metrics'] = clear_metrics()
                    section['level'] =  level
                    section['questions_answered'] = [0, 0, 0, 0, 0]
                    print(f"{name}'s data metrics for '{sub_topic}' has been reset.")
                    break
    print("level changed, database stats reset")
    # Write the updated data back to students.json
    post_ext_data(data, student_data_path)


In [ ]:
def get_student_subtopic_level(student, sub_topic):
    # Read the JSON file
    data = get_ext_data(student_data_path)

    # if we don't find the student, or the subtopic in the database, we will use the lowest level by default
    default_level = 1

    # Access the student's data from the database
    student_data = None
    for student_entry in data["students"]:
        if student in student_entry:
            student_data = student_entry[student]
            break

    if student_data is not None:
        # Find the sub-topic information for the student
        sub_topic_data = None
        for entry in student_data:
            if entry["sub_topic"] == sub_topic:
                sub_topic_data = entry
                break

    # Check if student exists in the database
    if student_data is None:
        print("student is not in database. They will be start at Level 1, Proficiency 1")
        return default_level
    # Check if sub-topic exists for the student
    if sub_topic_data is None:
        print(f"student has no data for '{sub_topic}' in this database. They will be start at Level 1.")
        return default_level
    else:
        # Retrieve the level and proficiency scores
        level = sub_topic_data["level"]

    # Return the level and proficiency scores
    return level

In [ ]:
def get_specific_metric_mistakes(specific_metric,metrics_data):
    related_mistakes = metrics_data[specific_metric]['related_mistakes']

    return related_mistakes

def get_all_metric_mistakes(metrics_data):
    metric_types = ["communication","interpretation", "computation","conceptual"]

    related_mistakes = []
    for metric_type in metric_types:
        # print(metric_type)
        specific_mistakes = get_specific_metric_mistakes(metric_type, metrics_data)
        related_mistakes.append(specific_mistakes)

    related_mistakes_str = f"{related_mistakes}"
    return related_mistakes_str
def get_all_student_related_mistakes(student, sub_topic):
    # Assuming you have the data dictionary defined
    data = get_ext_data(student_data_path)
    # Find the student in the data dictionary
    student_data = None
    for student_entry in data['students']:
        if student in student_entry:
            student_data = student_entry[student]
            break

    if student_data is None:
        return "Student not found"

    # Find the sub-topic in the student's data
    sub_topic_data = None
    for topic in student_data:
        if topic['sub_topic'] == sub_topic:
            sub_topic_data = topic
            break

    if sub_topic_data is None:
        return "Sub-topic not found for this student"

    metrics = sub_topic_data['proficiency_metrics']
    # Retrieve the related mistakes from the sub-topic data
    related_mistakes = get_all_metric_mistakes(metrics)
    # return metrics
    print(f"{student}'s previous mistakes with the sub_topic : {sub_topic}: {related_mistakes}\n")
    return related_mistakes

In [ ]:
# checks to see if the student has done well enough at a subtopic to move up a level
def is_level_update_needed(overall_avg_stats):
    bool = False
    avg_score, recent_scores =  overall_avg_stats["avg_score"],overall_avg_stats["recent_scores"]
    # if the avg score is 5, and we have 3 scores that make up the average, we need a level update
    if avg_score == 5 and len(recent_scores) == 3:
        bool = True
    return bool

# creates subtopic section with metric updates
def create_sub_topic_section(sub_topic, metric_updates):
    # make a new section
    new_section = {
        "sub_topic": sub_topic,
        "level": 1,
        "questions_answered": [0,0,0,0,0],
        "proficiency_metrics": clear_metrics()
    }
    # update the section with the metric updates
    new_section["proficiency_metrics"],new_section["level"],new_section["questions_answered"] = update_data(new_section,metric_updates)

    return new_section

# deletes a subtopic's metric data for user
def clear_metrics():
    metrics = {
        "overall_avg": {
            "avg_score": 0,
            "recent_scores": [
            ]
        },
        "communication": {
            "avg_score": 0,
            "related_mistakes": [
            ],
            "recent_scores": [
            ]
        },
        "interpretation": {
            "avg_score": 0,
            "related_mistakes": [],
            "recent_scores": []
        },
        "computation": {
            "avg_score": 5.0,
            "related_mistakes": [],
            "recent_scores": []
        },
        "conceptual": {
            "avg_score": 0,
            "related_mistakes": [],
            "recent_scores": []
        },
        "time": {
            "avg_score": 0,
            "avg_times": None,
            "recent_times": [
            ],
            "recent_scores": [
            ]
        }}
    return metrics

# helper functions for update_student_stats
# returns updated single metric array with the average value of the array
# recent_single_metric_score
# new_score: a score for one metric
def update_scores_and_average(recent_single_metric_scores, new_score):

    # the database stores the 3 most recent scores, so we will have to add our new_score and get rid of the old one

    # add new score
    # print(recent_single_metric_score)
    recent_single_metric_scores += [new_score]
    # remove the oldest score if there are more than 3 numbers in the list
    if len(recent_single_metric_scores) > 3:
        recent_single_metric_score = recent_single_metric_scores.pop(0)
        # recent_single_metric_score = recent_single_metric_scores[1:]
        # print(recent_single_metric_score)

    # get the new avg score and round to the last 2 decimals
    new_avg_score = np.mean(recent_single_metric_scores)
    new_avg_score = round(new_avg_score, 2)
    return recent_single_metric_scores,new_avg_score

# updata_data takes in a subtopic's data and metric_updates that need to be implemented
# returns the updated data, specifically the metrics, the level and the questions the student has answered
def update_data(data,metrics_updates):

    metrics, level,questions_answered  =  data["proficiency_metrics"], data["level"],data["questions_answered"]

    # update metrics
    metrics = update_metrics(metrics,metrics_updates)
    # update level and questions_answered

    # each index of the array corresponds to the amount of questions answered a a certain level of difficulty
    questions_answered[level-1] += 1

    # if we have to upgrade to the next level, we get rid of our previous level's stats
    if is_level_update_needed(metrics["overall_avg"]):
        # print(level)
        if level >= 5:
            topic = data["sub_topic"]
            print(f"Congratulations, you have mastered the topic: {topic} the highest level available. Please pick another topic to learn. ")
            # THe database will not reset and you will just keep all of your scores
        else:
            level += 1
            print(f"Congratulations, you have moved up to Level {level}")
            metrics = clear_metrics() # remove previous level's data


    return metrics,level,questions_answered

# updates all of the metrics for any subtopic
# takes in metrics and the updated from a recent evaluation
# returns updated metrics
def update_metrics(metrics, metric_updates):
    # print(f"\n\n metrics:{metrics} \n\n")
    # print(f"\n\n metrics updates:{metric_updates} \n\n")
    metric_types = ['overall_avg', 'communication', 'interpretation', 'computation', 'conceptual', 'time']

    # Iterate and update each metric
    for metric_type in metric_types:
        metric, metric_update = metrics[metric_type], metric_updates[metric_type]

        if metric_type == "overall_avg":
            new_score = metric_update

        else:
            # print(metric_update)
            new_score = metric_update["avg_score"]
            if 'related_mistakes' in metric_update:
                metric['related_mistakes'] = metric_update['related_mistakes']
            if 'seconds' in metric_update:
                new_time = metric_update['seconds']
                metric['recent_times'], metric['avg_times'] = update_scores_and_average(metric['recent_times'], new_time)

        # print(f"\n{metric}\n")
        # print(metric_type)
        metric['recent_scores'], metric['avg_score'] = update_scores_and_average(metric['recent_scores'], new_score)

    return metrics

In [ ]:
# checks to see if the student has done well enough at a subtopic to move up a level
def is_level_update_needed(overall_avg_stats):
    bool = False
    avg_score, recent_scores =  overall_avg_stats["avg_score"],overall_avg_stats["recent_scores"]
    # if the avg score is 5, and we have 3 scores that make up the average, we need a level update
    if avg_score == 5 and len(recent_scores) == 3:
        bool = True
    return bool

# creates subtopic section with metric updates
def create_sub_topic_section(sub_topic, metric_updates):
    # make a new section
    new_section = {
        "sub_topic": sub_topic,
        "level": 1,
        "questions_answered": [0,0,0,0,0],
        "proficiency_metrics": clear_metrics()
    }
    # update the section with the metric updates
    new_section["proficiency_metrics"],new_section["level"],new_section["questions_answered"] = update_data(new_section,metric_updates)

    return new_section

# deletes a subtopic's metric data for user
def clear_metrics():
    metrics = {
        "overall_avg": {
            "avg_score": 0,
            "recent_scores": [
            ]
        },
        "communication": {
            "avg_score": 0,
            "related_mistakes": [
            ],
            "recent_scores": [
            ]
        },
        "interpretation": {
            "avg_score": 0,
            "related_mistakes": [],
            "recent_scores": []
        },
        "computation": {
            "avg_score": 5.0,
            "related_mistakes": [],
            "recent_scores": []
        },
        "conceptual": {
            "avg_score": 0,
            "related_mistakes": [],
            "recent_scores": []
        },
        "time": {
            "avg_score": 0,
            "avg_times": None,
            "recent_times": [
            ],
            "recent_scores": [
            ]
        }}
    return metrics

# helper functions for update_student_stats
# returns updated single metric array with the average value of the array
# recent_single_metric_score
# new_score: a score for one metric
def update_scores_and_average(recent_single_metric_scores, new_score):

    # the database stores the 3 most recent scores, so we will have to add our new_score and get rid of the old one

    # add new score
    # print(recent_single_metric_score)
    recent_single_metric_scores += [new_score]
    # remove the oldest score if there are more than 3 numbers in the list
    if len(recent_single_metric_scores) > 3:
        recent_single_metric_score = recent_single_metric_scores.pop(0)
        # recent_single_metric_score = recent_single_metric_scores[1:]
        # print(recent_single_metric_score)

    # get the new avg score and round to the last 2 decimals
    new_avg_score = np.mean(recent_single_metric_scores)
    new_avg_score = round(new_avg_score, 2)
    return recent_single_metric_scores,new_avg_score

# updata_data takes in a subtopic's data and metric_updates that need to be implemented
# returns the updated data, specifically the metrics, the level and the questions the student has answered
def update_data(data,metrics_updates):

    metrics, level,questions_answered  =  data["proficiency_metrics"], data["level"],data["questions_answered"]

    # update metrics
    metrics = update_metrics(metrics,metrics_updates)
    # update level and questions_answered

    # each index of the array corresponds to the amount of questions answered a a certain level of difficulty
    questions_answered[level-1] += 1

    # if we have to upgrade to the next level, we get rid of our previous level's stats
    if is_level_update_needed(metrics["overall_avg"]):
        # print(level)
        if level >= 5:
            topic = data["sub_topic"]
            print(f"Congratulations, you have mastered the topic: {topic} the highest level available. Please pick another topic to learn. ")
            # THe database will not reset and you will just keep all of your scores
        else:
            level += 1
            print(f"Congratulations, you have moved up to Level {level}")
            metrics = clear_metrics() # remove previous level's data


    return metrics,level,questions_answered

# updates all of the metrics for any subtopic
# takes in metrics and the updated from a recent evaluation
# returns updated metrics
def update_metrics(metrics, metric_updates):
    # print(f"\n\n metrics:{metrics} \n\n")
    # print(f"\n\n metrics updates:{metric_updates} \n\n")
    metric_types = ['overall_avg', 'communication', 'interpretation', 'computation', 'conceptual', 'time']

    # Iterate and update each metric
    for metric_type in metric_types:
        metric, metric_update = metrics[metric_type], metric_updates[metric_type]

        if metric_type == "overall_avg":
            new_score = metric_update

        else:
            # print(metric_update)
            new_score = metric_update["avg_score"]
            if 'related_mistakes' in metric_update:
                metric['related_mistakes'] = metric_update['related_mistakes']
            if 'seconds' in metric_update:
                new_time = metric_update['seconds']
                metric['recent_times'], metric['avg_times'] = update_scores_and_average(metric['recent_times'], new_time)

        # print(f"\n{metric}\n")
        # print(metric_type)
        metric['recent_scores'], metric['avg_score'] = update_scores_and_average(metric['recent_scores'], new_score)

    return metrics

In [ ]:

# updates the student metrics in the database
def update_student_stats(name, sub_topic, metric_updates):
    # Get data from students.json
    data = get_ext_data(student_data_path)

    # Check if the student is in the database
    students = data["students"]
    for student in students:
        if name in student:
            # Check if the student has a section for the given subtopic
            sections = student[name]
            for section in sections:
                if section["sub_topic"] == sub_topic:
                    # Update the student's metrics
                    section["proficiency_metrics"], section["level"],section["questions_answered"], = update_data(section,metric_updates)
                    print(f"{name}'s data metrics for '{sub_topic} 'has been updated ")
                    break
            else:
                # update stats with cleared metric data
                new_section = create_sub_topic_section(sub_topic,metric_updates)
                # If the student does not have data for that subtopic, add metric_updates
                sections.append(new_section)
                print(f"{name}'s data metrics for '{sub_topic}' has been added ")

            break
    else:
        # If the student is not found in the database, create a new entry
        # update stats with cleared metric data
        new_section = create_sub_topic_section(sub_topic,metric_updates)
        students.append({
            name: [new_section]})
        print(f"{name}'s data metrics for '{sub_topic}'has been added ")
    # Write the updated data back to students.json
    post_ext_data(data, student_data_path)